In [2]:
import multiprocessing

if __name__ == '__main__':
    multiprocessing.set_start_method('forkserver')

In [3]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score

import datetime
import matplotlib
from matplotlib import pyplot

/Users/scottcha/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
from fastai.imports import *
from fastai.structured import *

In [14]:
prediction_trend_data = [] 
input_numeric_columns = []
input_categorical_columns = []
output_columns = []

normalize = False
dfOriginal = pd.read_csv('../Data/V1.1FeaturesWithLabelsTestRegion220131201To20180430.csv', parse_dates=['Date'], low_memory=False)
df = dfOriginal
all_columns = list(df)
numeric_regex = re.compile("n_*")
input_numeric_columns = list(filter(numeric_regex.match, all_columns))
categorical_regex = re.compile("c_*")                               
input_categorical_columns = list(filter(categorical_regex.match, all_columns))
output_regex = re.compile("o_*")                               
output_columns = list(filter(output_regex.match, all_columns))
metadata_columns = ["Lat", "Lon", "UnifiedRegion", "Date"]
df = df[input_numeric_columns+input_categorical_columns+output_columns+prediction_trend_data+metadata_columns]
    

In [15]:
df = df[df['o_Day1DangerAboveTreeline']!='no-data']
df = df[df['o_Day1DangerNearTreeline']!='no-data']
df = df[df['o_Day1DangerBelowTreeline']!='no-data']

df = df[df['o_Day1DangerAboveTreeline']!='Extreme']
df = df[df['o_Day1DangerNearTreeline']!='Extreme']
df = df[df['o_Day1DangerBelowTreeline']!='Extreme']

In [16]:
#clean mising values
mappingMissingValues = {-9999:np.nan}
df = df.replace(mappingMissingValues)

df = df.interpolate(method='linear', axis=0).ffill().bfill()


In [18]:
train_cats(df)
df['o_Day1DangerAboveTreeline'].cat.set_categories(['High', 'Considerable', 'Moderate', 'Low'], ordered=True, inplace=True)

In [19]:
splitBySeason = True
yColumns = ['o_Day1DangerAboveTreeline']
yNearColumn = ['o_Day1DangerNearTreeline']
yBelowColumn = ['o_Day1DangerBelowTreeline']

if(splitBySeason):
    df_train = df[df['Date'] < datetime.datetime(2017, 5, 1) ] 
    df_test = df[df['Date'] > datetime.datetime(2017, 5, 1)]
    
    y_Above_train = df_train[yColumns + metadata_columns]
    y_Near_train = df_train[yNearColumn + metadata_columns]
    y_Below_train = df_train[yBelowColumn + metadata_columns]
    
    y_Above_test = df_test[yColumns + metadata_columns]
    y_Near_test = df_test[yNearColumn + metadata_columns]
    y_Below_test = df_test[yBelowColumn + metadata_columns]
    
    #feature engineer date
    add_datepart(df_train, 'Date')
    add_datepart(df_test, 'Date')
    date_columns = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed']
    
    #same x input trained to get a different output
    X_Above_train = X_Near_train = X_Below_train = df_train[input_numeric_columns+input_categorical_columns+date_columns]
    X_Above_test = X_Near_test = X_Below_test = df_test[input_numeric_columns+input_categorical_columns+date_columns]
    
   
    
else:
    #random stratified split
    X = df[input_numeric_columns+input_categorical_columns]
    
    yAbove=df[yColumns + metadata_columns]
    X_Above_train, X_Above_test, y_Above_train, y_Above_test = train_test_split(X, yAbove, stratify=yAbove["o_Day1DangerAboveTreeline"], test_size=0.20, random_state=1)
    
    yNear=df[yNearColumn+metadata_columns]
    X_Near_train, X_Near_test, y_Near_train, y_Near_test = train_test_split(X, yNear, stratify=yNear['o_Day1DangerNearTreeline'], test_size=0.20, random_state=1)
    
    yBelow=df[yBelowColumn+metadata_columns]
    X_Below_train, X_Below_test, y_Below_train, y_Below_test = train_test_split(X, yBelow, stratify=yBelow['o_Day1DangerBelowTreeline'], test_size=0.20, random_state=1)

/Users/scottcha/anaconda/envs/fastai/lib/python3.6/site-packages/fastai/structured.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
/Users/scottcha/anaconda/envs/fastai/lib/python3.6/site-packages/fastai/structured.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
/Users/scottcha/anaconda/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a co

In [ ]:
#this uses median to fill missing values, we want to interpolate
#X_Above_train2, y_Above_train2, nas = proc_df(X_Above_train, 'o_Day1DangerAboveTreeline')

In [ ]:
#X_Above_test2, y_Above_test2, nas = proc_df(X_Above_test, 'o_Day1DangerAboveTreeline')

In [20]:
X_Above_test.head()

,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0.1250,2.255751,0.2500,1.962264,0.4375,1.868442,0.5625,2.046617,0.6875,2.033102,...,29,6,119,False,False,False,False,False,False,1524960000
1,0.3750,2.797099,0.8750,2.399905,1.3750,2.350245,2.1875,2.317809,2.7500,2.315767,...,29,6,119,False,False,False,False,False,False,1524960000
2,0.2500,2.745156,0.7500,2.320603,1.1875,2.237345,1.8125,2.214742,2.8125,2.185103,...,29,6,119,False,False,False,False,False,False,1524960000
3,0.2500,2.602469,0.6875,2.246537,1.1250,2.149259,1.6250,2.175206,2.5625,2.130727,...,29,6,119,False,False,False,False,False,False,1524960000
4,0.9375,1.471069,2.3750,1.003212,3.3750,0.438343,4.0625,1.140340,4.3750,1.678012,...,29,6,119,False,False,False,False,False,False,1524960000


In [23]:
m = RandomForestClassifier(n_jobs = -1,
                           oob_score=True,
                           max_features="sqrt",
                           n_estimators=180)
m.fit(X_Above_train, y_Above_train[yColumns])
m.score(X_Above_train, y_Above_train[yColumns].values.ravel())

/Users/scottcha/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.992386603511301

In [ ]:
tmp = X_Above_test2[X_Above_train2.columns]

In [26]:
X_Above_test.head()

,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0.1250,2.255751,0.2500,1.962264,0.4375,1.868442,0.5625,2.046617,0.6875,2.033102,...,29,6,119,False,False,False,False,False,False,1524960000
1,0.3750,2.797099,0.8750,2.399905,1.3750,2.350245,2.1875,2.317809,2.7500,2.315767,...,29,6,119,False,False,False,False,False,False,1524960000
2,0.2500,2.745156,0.7500,2.320603,1.1875,2.237345,1.8125,2.214742,2.8125,2.185103,...,29,6,119,False,False,False,False,False,False,1524960000
3,0.2500,2.602469,0.6875,2.246537,1.1250,2.149259,1.6250,2.175206,2.5625,2.130727,...,29,6,119,False,False,False,False,False,False,1524960000
4,0.9375,1.471069,2.3750,1.003212,3.3750,0.438343,4.0625,1.140340,4.3750,1.678012,...,29,6,119,False,False,False,False,False,False,1524960000


In [27]:
m.score(X_Above_test, y_Above_test[yColumns].values.ravel())

0.6210892236384704

In [ ]:
import matplotlib.pyplot as plt

from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# Author: Kian Ho <hui.kian.ho@gmail.com>
#         Gilles Louppe <g.louppe@gmail.com>
#         Andreas Mueller <amueller@ais.uni-bonn.de>
#
# License: BSD 3 Clause

print(__doc__)

RANDOM_STATE = 123

# Generate a binary classification dataset.
#X, y = make_classification(n_samples=500, n_features=25,
#                           n_clusters_per_class=1, n_informative=15,
#                           random_state=RANDOM_STATE)
X = X_Above_train2
y = y_Above_train2

# NOTE: Setting the `warm_start` construction parameter to `True` disables
# support for parallelized ensembles but is necessary for tracking the OOB
# error trajectory during training.
ensemble_clfs = [
    ("RandomForestClassifier, max_features='sqrt'",
        RandomForestClassifier(n_estimators=100,
                               warm_start=True, oob_score=True,
                               max_features="sqrt",
                               n_jobs = 3,
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features='log2'",
        RandomForestClassifier(n_estimators=100,
                               warm_start=True, max_features='log2',
                               n_jobs = 3,
                               oob_score=True,
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features=None",
        RandomForestClassifier(n_estimators=100,
                               warm_start=True, max_features=None,
                               n_jobs=3,
                               oob_score=True,
                               random_state=RANDOM_STATE))
]

# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

# Range of `n_estimators` values to explore.
min_estimators = 15
max_estimators = 175

for label, clf in ensemble_clfs:
    i = min_estimators
    for i in range(min_estimators, max_estimators + 1, 10):
        clf.set_params(n_estimators=i)
        clf.fit(X, y)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

In [ ]:
best_preds = [np.argmax(line) for line in resultAbove]

In [ ]:
best_preds_df = pd.DataFrame(best_preds, columns=['o_Day1DangerAboveTreeline'])

In [ ]:
best_preds_df.head()

In [ ]:
mapping = {0:'Considerable', 1:'High', 2:'Low', 3:'Moderate'}
best_preds_df = best_preds_df.replace({'o_Day1DangerAboveTreeline': mapping})
best_preds_df['o_Day1DangerAboveTreeline'].head()

In [ ]:
evaluateSingleClassShort(y_Above_test['o_Day1DangerAboveTreeline'], best_preds_df)
cnf_matrix = confusion_matrix(y_Above_test['o_Day1DangerAboveTreeline'], best_preds_df)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'], normalize=True,
                      title='Normalized confusion matrix')
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'],
                      title='Confusion matrix, without normalization')
plt.show()

In [ ]:
def evaluateSingleClass(y_test, result):
    cm = confusion_matrix(y_test, result)
    print(cm)
    accuracy = accuracy_score(y_test.values, result)
    print("Accuracy: " + str(accuracy))
    print("Classification Report")
    print(classification_report(y_test.values, result))

def evaluateSingleClassShort(y_test, result):
    accuracy = accuracy_score(y_test.values, result)
    print("Accuracy: " + str(accuracy))
    print("Classification Report")
    print(classification_report(y_test.values, result))

#nice confustion matrix plot taken from scikit-learn docs
import itertools
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
importances = pd.Series(gbmgpu.feature_importances_)
out = pd.concat([pd.Series(X_Above_train.columns), importances], axis=1)

In [ ]:
out.columns = ['feature', 'importance']

In [ ]:
out.sort_values('importance',ascending=False)[:100]